In [1]:
import imageio
from print_nanny_webapp.utils.predictor import ThreadLocalPredictor, predict_events_to_dataframe
import pandas as pd
import tensorflow as tf
import numpy as np
import tempfile
from celery import chord, chain

from print_nanny_webapp.alerts.tasks import prediction_dicts_to_dataframe, predict_postprocess_frame, calc_metrics

In [2]:
reader = imageio.get_reader('/app/data/-0-0uBnUSnc.mp4')



CHUNKS=10
temp_dir = tempfile.mkdtemp()
grouped = predict_postprocess_frame.chunks( 
    ((i, frame, temp_dir) for i, frame in enumerate(reader))
    , CHUNKS).group() 
    

chord1 = chord(grouped, prediction_dicts_to_dataframe.s())
df = chord1().get()
df

,detection_boxes,detection_classes,detection_scores,num_detections
id,,,,
0,"[[0.45823687, 0.59046566, 0.4821356, 0.6149084...","[1, 1, 1, 5, 1, 1, 5, 1, 1, 5, 1, 4, 1, 5, 5, ...","[0.4609375, 0.171875, 0.12890625, 0.0625, 0.04...",40.0
1,"[[0.48223186, 0.84138817, 0.6471373, 0.9972691...","[5, 3, 5, 3, 3, 5, 1, 1, 4, 4, 4, 4, 3, 5, 5, ...","[0.3125, 0.1484375, 0.12890625, 0.09765625, 0....",40.0
2,"[[0.51865387, 0.45078614, 0.55876267, 0.483172...","[1, 5, 5, 5, 4, 5, 5, 1, 5, 5, 1, 4, 1, 5, 1, ...","[0.75, 0.4609375, 0.34765625, 0.12890625, 0.07...",40.0
3,"[[0.5182006, 0.45123178, 0.5565435, 0.48219222...","[1, 5, 5, 5, 1, 4, 1, 5, 5, 4, 5, 1, 5, 4, 1, ...","[0.71875, 0.4609375, 0.25, 0.09765625, 0.09765...",40.0
4,"[[0.5202146, 0.4506033, 0.5598745, 0.4812172],...","[1, 4, 5, 4, 5, 1, 1, 5, 4, 5, 1, 4, 1, 1, 5, ...","[0.75, 0.4609375, 0.09765625, 0.09765625, 0.08...",40.0
...,...,...,...,...
278,"[[0.54678464, 0.22103597, 0.9956572, 0.6905802...","[3, 4, 1, 3, 3, 4, 4, 3, 3, 1, 1, 4, 3, 4, 4, ...","[0.8515625, 0.8046875, 0.578125, 0.0625, 0.039...",40.0
279,"[[0.5363299, 0.21297379, 0.99542195, 0.6772625...","[3, 4, 1, 3, 4, 3, 4, 3, 3, 4, 3, 3, 3, 3, 4, ...","[0.8046875, 0.75, 0.3828125, 0.046875, 0.04687...",40.0
280,"[[0.54395175, 0.21836348, 0.9878001, 0.6879077...","[3, 4, 1, 3, 4, 4, 3, 3, 4, 3, 3, 3, 4, 1, 5, ...","[0.8046875, 0.71875, 0.421875, 0.07421875, 0.0...",40.0


In [3]:

framerate = reader.get_meta_data()['fps']


multi_df, fail_df, confident_df = calc_metrics(df, framerate)


multi_df


detection_classes detection_scores     timecode
frame_id label                                                    
0        nozzle                    1         0.460938  00:00:00:00
         nozzle                    1         0.171875  00:00:00:00
         nozzle                    1         0.128906  00:00:00:00
         raft                      5           0.0625  00:00:00:00
         nozzle                    1         0.046875  00:00:00:00
...                              ...              ...          ...
282      print                     4        0.0078125  00:00:11:07
         nozzle                    1        0.0078125  00:00:11:07
         spaghetti                 3        0.0078125  00:00:11:07
         spaghetti                 3        0.0078125  00:00:11:07
         spaghetti                 3        0.0078125  00:00:11:07

[11320 rows x 3 columns]

In [4]:
# from print_nanny_webapp.alerts.tasks import plot_timelapse_graphs
# plot_timelapse_graphs(df, framerate, 1, '/tmp/')

In [5]:
fail_df


detection_classes detection_scores     timecode
frame_id label                                                    
147      spaghetti                 3         0.617188  00:00:05:22
149      spaghetti                 3          0.71875  00:00:05:24
         adhesion                  2         0.617188  00:00:05:24
150      spaghetti                 3         0.871094  00:00:06:00
         adhesion                  2           0.6875  00:00:06:00
...                              ...              ...          ...
278      spaghetti                 3         0.851562  00:00:11:03
279      spaghetti                 3         0.804688  00:00:11:04
280      spaghetti                 3         0.804688  00:00:11:05
281      spaghetti                 3         0.804688  00:00:11:06
282      spaghetti                 3          0.78125  00:00:11:07

[188 rows x 3 columns]

In [6]:
confident_df

detection_classes detection_scores     timecode
frame_id label                                                    
2        nozzle                    1             0.75  00:00:00:02
3        nozzle                    1          0.71875  00:00:00:03
4        nozzle                    1             0.75  00:00:00:04
5        nozzle                    1          0.71875  00:00:00:05
         print                     4         0.539062  00:00:00:05
...                              ...              ...          ...
280      print                     4          0.71875  00:00:11:05
281      spaghetti                 3         0.804688  00:00:11:06
         print                     4          0.71875  00:00:11:06
282      spaghetti                 3          0.78125  00:00:11:07
         print                     4             0.75  00:00:11:07

[824 rows x 3 columns]

In [9]:

pd.options.plotting.backend = "plotly"
confident_df.reset_index().plot(
    x='label',
    y='detection_scores',
    kind='box',
    title="Print Nanny's Confidence Distribution by Label",
    color='label',
)

In [10]:
g = confident_df.reset_index() #.set_index('frame_id')
import plotly.express as px
from scipy import signal
y = signal.savgol_filter(g['detection_scores'], int(framerate), 3)



fig = px.line(
    g, x='frame_id', y=y, color='label', facet_col='label', line_group='label',
    height=800, width=800, facet_col_wrap=1, facet_col_spacing=0.05
)

fig.update_yaxes(title_text='Confidence')
#fig.update_xaxes(title_text='Time')

fig.update_layout(
    overwrite=True,
    title_text="Print Nanny Confidence over Time",
    xaxis_title='Time',
    legend_title='Detection'
)

fig.show()


In [18]:
import plotly.graph_objects as go

fig = go.Figure()

print_trace = g[g['label'] == 'print']
fail_trace = fail_df.reset_index()


print_rescaled = print_trace['detection_scores'] - print_trace['detection_scores'].min()
print_rescaled = print_rescaled / print_trace['detection_scores'].max()

fig.add_trace(go.Scatter(
    x=print_trace['frame_id'], 
    y=signal.savgol_filter(print_trace['detection_scores'],int(framerate), 6),
    fill=None,
    mode='lines',
    name='print'

))

y_rescaled = fail_trace['detection_scores'] - fail_trace['detection_scores'].min()
y_rescaled = y_rescaled / fail_trace['detection_scores'].max()

fig.add_trace(go.Scatter(
    x=fail_trace['frame_id'], 
    y=signal.savgol_filter(
        fail_trace['detection_scores'],     int(framerate), 3),
    fill=None,
    mode='lines',
    cliponaxis=False,
    name='defects',    
))

fig.update_layout(
    overwrite=True,
    title_text="Print Health Scores (Absolute)",
    xaxis_title='Time',
    yaxis_title='Health Score'
)


fig.show()
print_trace.groupby('frame_id')['detection_scores'].sum() - fail_trace.groupby('frame_id')['detection_scores'].sum()

frame_id
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
         ...   
278   -0.046875
279   -0.054688
280   -0.085938
281   -0.085938
282   -0.031250
Name: detection_scores, Length: 256, dtype: float64

In [ ]:
import plotly.graph_objects as go

split_df = pd.concat({'fail': fail_trace, 'print': print_trace}).reset_index()

mask = split_df.level_0 == 'fail'

y = split_df[~mask].groupby('timecode')['detection_scores'].sum().subtract(
    split_df[mask].groupby('timecode')['detection_scores'].sum(),
    fill_value=0
)
# y['timecode'] = y.apply(lambda x: frames_to_timecode(x.name), axis=1)
#split_df.loc[mask, 'detection_scores'] = -split_df[mask]['detection_scores'] 
fig = go.Figure(go.Waterfall(
    orientation = "v",
    x = y.index,
    y =  y,
    #color='level_0'
    #selected_points = 1

))

alert_offset = int(framerate/2)
fig.update_layout(
    overwrite=True,
    title_text="Print Health Over Time",
    xaxis_title='Time',
    yaxis_title='Health Score'
)

fig.update_xaxes(
    nticks=20
)

fig.add_annotation(
    x=y[y < 0].index[alert_offset],
    y=1,
    text="Print Nanny alerts you at this time",
    showarrow=True,
    arrowhead=1
)

fig.add_vrect(
    x0=y[y < 0].index[alert_offset], x1=y[y < 0].index[-1],
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
)

fig.show()

In [ ]:
y[y<0].sum()

In [ ]:
y[y>0].sum()

In [ ]:
multi_df.to_csv()